In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

# Spin alignment

In [ ]:
import logging

import graphviz
import qrules
import sympy as sp
from IPython.display import Math

import ampform
from ampform.helicity import formulate_wigner_d

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)


def show_transition(transition, **kwargs):
    if "size" not in kwargs:
        kwargs["size"] = 5
    dot = qrules.io.asdot(transition, **kwargs)
    display(graphviz.Source(dot))

## Helicity formalism

Imagine we want to formulate the amplitude for the following **single** {class}`~qrules.transition.StateTransition`:

In [ ]:
full_reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["K0", ("Sigma+", [+0.5]), ("p~", [+0.5])],
    allowed_intermediate_particles=["Sigma(1660)~-", "N(1650)+"],
    allowed_interaction_types="strong",
    formalism="helicity",
)
graphs = full_reaction.to_graphs()
single_transition_reaction = full_reaction.from_graphs(
    [graphs[0]], formalism=full_reaction.formalism
)
show_transition(single_transition_reaction)

The specific {attr}`~qrules.transition.State.spin_projection`s for each {attr}`~qrules.transition.State.particle` only make sense _given a specific reference frame_. AmpForm's {class}`.HelicityAmplitudeBuilder` interprets these projections as the **helicity** $\lambda=\vec{S}\cdot\vec{p}$ of each particle _in the rest frame of the initial state_ $J/\psi$.

Ignoring dynamics and coefficients, the {class}`.HelicityModel` for this single transition is rather simple:

In [ ]:
builder = ampform.get_builder(single_transition_reaction)
model = builder.formulate()
model.expression.subs(model.parameter_defaults).subs(1.0, 1)

The two Wigner-$D$ functions come from the two **two-body decay nodes** that appear in the {class}`~qrules.transition.StateTransition` above. They were formulated with {func}`.formulate_wigner_d`:

In [ ]:
transition = single_transition_reaction.transitions[0]
formulate_wigner_d(transition, node_id=0) * formulate_wigner_d(
    transition, node_id=1
)

Now, as {func}`.formulate_wigner_d` explains, the numbers that appear in the Wigner-$D$ functions here are computed from the helicities of the decay products. But there's a subtle problem with this: these helicities are _assumed to be in the rest frame of each parent particle_. For the first node, this is fine, because the parent particle rest frame matches that of the initial state in the {class}`~qrules.transition.StateTransition` above. In the second node, however, we are in a different rest frame.

When summing over all amplitudes in the complete {class}`~qrules.transition.StateTransitionCollection` that contains all spin projections (helicities), the mismatch in rest frames evens out and the problem we identified here can be ignored. It again becomes a problem, however, when we are formulating an amplitude model _with different topologies_. An example would be the following reaction:

In [ ]:
show_transition(full_reaction, collapse_graphs=True)

<!-- cspell:ignore mikhasenko Dalitzplot Threebody -->
The {class}`.HelicityAmplitudeBuilder` implements the 'standard' helicity formalism as described in {cite}`richmanExperimenterGuideHelicity1984, kutschkeAngularDistributionCookbook1996, chungSpinFormalismsUpdated2014` and simply sums over the different amplitudes to get the full amplitude:

In [ ]:
builder = ampform.get_builder(full_reaction)
model = builder.formulate()
latex = sp.multiline_latex(
    sp.Symbol("I"),
    model.expression.subs(model.parameter_defaults).subs(1.0, 1),
    environment="eqnarray",
)
Math(latex)

As pointed out in {cite}`marangottoHelicityAmplitudesGeneric2020, mikhasenkoDalitzplotDecompositionThreebody2020, wangNovelMethodTest2020`, this is wrong because of the mismatch in reference frames for the helicities.

## Aligning reference frames

In what follows, we follow {cite}`marangottoHelicityAmplitudesGeneric2020` to align all amplitudes in the different topologies back to the initial state reference frame $A$, so that they can be correctly summed up. Specifically, we want to formulate a new, correctly aligned amplitude $\mathcal{A}^{A\to 0,1,\dots}_{m_A,m_0,m_1,\dots}$ from the original amplitudes $\mathcal{A}^{A\to R,S,i,...\to 0,1,\dots}_{\lambda_A,\lambda_0,\lambda_1,\dots}$ by applying Eq.(45) and Eq.(47) for generic, multi-body decays.